<a href="https://colab.research.google.com/github/aguscura/UBA-data-science/blob/main/tp1_c3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTANTE

**Obviamente me mapea reviews escritas en ingles a "portugues" o "español", porque no le "enseñé" un texto en ingles. Se me complica conseguir las reviews, no encuentro scores bajos cercanos a cero. Agrego texto en ingles y tampoco logro encontrar la solucion. Tome distintos samples y en ningun caso logro resolver.**

In [127]:
# from google.colab import files
# files.upload()

In [128]:
import PyPDF2
import re

pdfFileObj = open('/content/spn.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

esp_text = ""

for i in range (0,pdfReader.numPages):
  
  page = pdfReader.getPage(i)
  esp_text += page.extractText()

esp_list = re.split('\n',esp_text)
esp_list[1]


'Declaración Universal de Derechos Humanos '

In [129]:
pdfFileObj = open('/content/declaracion portugues.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

por_text = ""

for i in range (0,pdfReader.numPages):
  
  page = pdfReader.getPage(i)
  por_text += page.extractText()

por_list = re.split('\n',por_text)
por_list[6]


'Todos os seres humanos nascem livres e iguais em dignidade e em direitos. Dotados de razão e de consciência, devem agir uns para com os outros em espírito de fraternidade.  '

In [180]:
pdfFileObj = open('/content/english.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

en_text = ""

for i in range (0,pdfReader.numPages):
  
  page = pdfReader.getPage(i)
  en_text += page.extractText()

en_list = re.split('\n',en_text)
en_list[7]

'any form by any means—electronic, mechanical, photocopy, recording, or'

In [130]:
# !pip install zstandard

from io import BytesIO
import PIL.Image
import numpy as np
import gzip
import random
import lzma
import os
from tqdm import tqdm
import zstandard
import bz2
from functools import lru_cache
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [131]:
@lru_cache(maxsize=10000)
def compress_size(bytefile):
    return len(zstandard.compress(bytefile))

def ncd(obj1, obj2):
    obj1 = bytes(obj1, 'utf-8')
    obj2 = bytes(obj2, 'utf-8')
    len1 = compress_size(obj1)
    len2 = compress_size(obj2)
    concat_len = len(zstandard.compress(obj1+obj2))
    return (concat_len-min(len1, len2))/max(len1, len2)

![](botnet.png "")

# Entreno con el 100% de las declaraciónes. Mi "test set" van a ser las reviews

In [132]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(esp_list+por_list, 
#                                                     ["Español"]*len(esp_list)+["Portugues"]*len(por_list), 
#                                                     test_size=0.1, random_state=42)

In [182]:
X_train = esp_list + por_list + en_list
y_train = ["Español"]*len(esp_list) + ["Portugues"]*len(por_list) + ["Inglés"]*len(en_list)

# BUSCO TEST

In [135]:
# !gdown "1okxCrPipVJmexBSVZ34kJlttw02-7kTX"

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)
Downloading...
From: https://drive.google.com/uc?id=1okxCrPipVJmexBSVZ34kJlttw02-7kTX
To: /content/review.csv
100% 2.43G/2.43G [00:18<00:00, 132MB/s]


In [144]:
import pandas as pd
df = pd.read_csv("/content/review.csv")
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
2,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4.0,1.0,0.0,1.0,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07
3,-P5E9BYUaK7s3PwBF5oAyg,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5.0,0.0,0.0,0.0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05
4,YbMyvlDA2W3Py5lTz8VK-A,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5.0,0.0,0.0,0.0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34
...,...,...,...,...,...,...,...,...,...
3838100,zHZ-A1qyKDEgyZMDaD--wg,_XVdmFWSgTN6YlojUxixTA,6WaI-IN8ql0xpEKlb4q8tg,5.0,1.0,0.0,0.0,We redesigned my moms dress and mad it complet...,2022-01-17 20:59:01
3838101,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5.0,2.0,1.0,2.0,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
3838102,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4.0,1.0,0.0,0.0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
3838103,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5.0,1.0,0.0,0.0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [183]:
X_test = df["text"].tolist()
X_test = X_test[20000:30000]
len(X_test)

10000

# Infiero

In [184]:
from collections import Counter
preds = []

for testfrase in tqdm(X_test):
   
    distances = [ncd(frase, testfrase) for frase in X_train]
    preds.append(Counter(np.asarray(y_train)[np.argsort(distances)[:3]]).most_common(1)[0][0])

100%|██████████| 10000/10000 [04:57<00:00, 33.65it/s]


In [187]:
df2 = pd.DataFrame(list(zip(X_test, preds, distances)),
               columns =['Review', 'Predicción', 'Distances'])

In [191]:
portugues = df2[df2["Predicción"] == "Portugues"].sort_values(by=['Distances'],ascending=True)
portugues

,Review,Predicción,Distances
1316,I've heard some really great things about Hive...,Portugues,0.636364
1205,Went there tonight for the first time with a p...,Portugues,0.652893
205,So ive been to place several times over the ye...,Portugues,0.661157
459,We loved it. 2nd time. Food fresh and yummy. ...,Portugues,0.661157
942,Right across the street from Independence Hall...,Portugues,0.677686
...,...,...,...
876,I was very disappointed in my experience at Ma...,Portugues,0.966942
526,Don't be deterred by the line. It moves fast a...,Portugues,0.966942
971,We had the pleasure of dining here for dinner ...,Portugues,0.966942
107,The best Italian restaurant in the city. You c...,Portugues,0.973577


In [192]:
español = df2[df2["Predicción"] == "Español"].sort_values(by=['Distances'],ascending=True)
español

,Review,Predicción,Distances
1189,"Kristen has been 100% phenomenal, hands down. ...",Español,0.644628
708,We were in town for the weekend and this place...,Español,0.669421
1436,I admit I was a little shy of the adventurous ...,Español,0.669421
1184,I'm really surprised that someone would take t...,Español,0.669421
829,"What does a one $ sign, fabulous food in a ple...",Español,0.677686
...,...,...,...
1538,We are from MN and I saw this place on social ...,Español,0.958678
390,First time at Honeygrow today. My client reque...,Español,0.958678
1515,Everything here is super good and relatively a...,Español,0.966942
494,I got sushi and vegetables noodles for takeout...,Español,0.966942


In [141]:
# accuracy_score(y_test, preds)

In [142]:
# print(classification_report(y_test, preds))